In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from shapely.geometry import Point
from shapely.ops import transform
import pyproj
from functools import partial
from collections import Counter
import pickle

In [3]:
df_pois = pd.read_csv('./pois.csv')

In [4]:
def gera_isocota(ponto, raio):
    raio_ponto_de_estudo_utm = lat_lng_to_utm(ponto).buffer(raio)
    raio_ponto_de_estudo_lat_lng = utm_to_lat_lng(raio_ponto_de_estudo_utm)
    return raio_ponto_de_estudo_lat_lng


def lat_lng_to_utm(lat_lng_geom):
     project = partial(
     pyproj.transform,
     pyproj.Proj(init='epsg:4326'), # source coordinate system
     pyproj.Proj(init='epsg:3857')) # destination coordinate system
     utm_geom = transform(project,lat_lng_geom)
     return utm_geom

def utm_to_lat_lng(utm_geom):
     project = partial(
     pyproj.transform,
     pyproj.Proj(init='epsg:3857'), # source coordinate system
     pyproj.Proj(init='epsg:4326')) # destination coordinate system
     lat_lng_geom = transform(project,utm_geom)
     return lat_lng_geom
    
def obtem_configuracao():
    return {'faculdades','escolas', 'ponto_de_onibus', 'concorrentes__grandes_redes',
            'concorrentes__pequeno_varejista', 'minhas_lojas', 'agencia_bancaria', 'padaria',
            'acougue', 'restaurante', 'correio', 'loterica'}

In [5]:
dic_enriquecido = {}                                                                                                                       
target = 'minhas_lojas'
area_de_influencia = 1000
for row_target in df_pois.loc[df_pois['tipo_POI'] == target].itertuples():
        lat, lng = row_target.latitude, row_target.longitude
        pt = Point(lng, lat)
        isocota = gera_isocota(pt, area_de_influencia)
        min_lng, min_lat, max_lng, max_lat = isocota.bounds
        mask = (df_pois.latitude.between(min_lat, max_lat) &
                df_pois.longitude.between(min_lng, max_lng))
        counter = Counter()
        for row_poi in df_pois.loc[mask].itertuples():
            if Point(row_poi.longitude, row_poi.latitude).within(isocota):
                counter.update([row_poi.tipo_POI])
        dic_enriquecido[row_target.Index] = counter
df_data = pd.DataFrame.from_dict(dic_enriquecido, orient='Index').fillna(0)

for i in  obtem_configuracao():
    if i not in df_data:
        df_data[i] = 0                                                                                                                        
                                                                                                                                               
              

c:\Users\danil\Documents\Case\venv-prod-camp\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\danil\Documents\Case\venv-prod-camp\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\danil\Documents\Case\venv-prod-camp\Lib\site-packages\shapely\ops.py:274: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.

In [6]:
df_faturamento = pd.read_csv('./faturamento.csv')

In [7]:
df_model_data = df_data.join(df_faturamento.set_index('id'))
df_model_data = df_model_data[sorted(df_model_data.columns)]

In [8]:
df_model_data

,acougue,agencia_bancaria,concorrentes__grandes_redes,concorrentes__pequeno_varejista,correio,escolas,faculdades,faturamento_medio_mensal,loterica,minhas_lojas,padaria,ponto_de_onibus,restaurante
5061,16,33,1.0,1.0,2.0,3.0,4.0,8497.218364,3.0,2,34,8.0,5
5062,10,27,0.0,6.0,4.0,4.0,2.0,6649.221857,3.0,1,24,7.0,8
5063,11,26,0.0,2.0,5.0,10.0,3.0,11382.550139,3.0,2,32,4.0,6
5064,6,18,0.0,3.0,5.0,5.0,1.0,5797.611914,4.0,2,24,7.0,8
5065,16,25,0.0,2.0,2.0,7.0,6.0,11993.147033,3.0,2,36,10.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5343,15,36,0.0,0.0,5.0,3.0,0.0,7216.154585,5.0,2,21,6.0,6
5348,21,22,0.0,2.0,3.0,3.0,0.0,5995.309638,6.0,1,22,8.0,7
5129,22,26,0.0,4.0,2.0,0.0,0.0,4562.864635,3.0,2,29,5.0,6
5287,19,27,0.0,3.0,2.0,0.0,0.0,5694.647434,3.0,2,34,4.0,6


In [9]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
X, y = df_model_data.drop(['faturamento_medio_mensal'], axis=1), df_model_data['faturamento_medio_mensal']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
cross_val_score(model, X_train, y_train, scoring='r2', cv=5)

array([0.75557166, 0.73695078, 0.72754761, 0.83573728, 0.85396801])

In [10]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
r2_score(y_test, y_pred)

0.7400201899546334

In [11]:
model.fit(X, y)

RandomForestRegressor(random_state=42)

In [12]:
with open('./model_campifarma.pickle', 'wb') as p:
    pickle.dump(model, p)